In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Patel\\OneDrive\\Desktop\\pl\\chest classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Patel\\OneDrive\\Desktop\\pl\\chest classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size: list
    params_include_top: bool
    params_weights: str
    params_learning_rate: float
    params_classes : int

In [6]:
from cnnclassifer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnclassifer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        print( self.config.prepare_base_model)

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir= Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size= self.params.IMAGE_SIZE,
            params_include_top= self.params.INCLUDE_TOP,
            params_weights= self.params.WEIGHTS,
            params_learning_rate= self.params.LEARNING_RATE,
            params_classes = self.params.CLASSES,
        )

        return prepare_base_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import json
import numpy as np

In [9]:
class PrepareBaseModel:
    def __init__(self,config=PrepareBaseModelConfig):
        self.config=config
    
    def get_base_model(self):
        self.model = tf.keras.applications.EfficientNetB0(
        input_shape=self.config.params_image_size,
        weights=self.config.params_weights,      # usually "imagenet"
        include_top=self.config.params_include_top  # False if you’re adding your own classifier
    )

        self.save_model(path=str(self.config.base_model_path),model=self.model)

    @staticmethod
    def save_model(path: str, model: tf.keras.Model):
        # Solution 1: Use SavedModel format (recommended)
        try:
            model.save(path, save_format='tf')
        except Exception as e:
            print(f"SavedModel format failed: {e}")
            # Solution 2: Use HDF5 format as fallback
            try:
                if not path.endswith('.h5'):
                    path = path + '.h5'
                model.save(path, save_format='h5')
            except Exception as e2:
                print(f"HDF5 format also failed: {e2}")
                # Solution 3: Save weights only
                model.save_weights(path + '_weights.h5')
                print(f"Saved weights only to {path}_weights.h5")

    @staticmethod    
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
        x = tf.keras.layers.Dense(128, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        prediction = tf.keras.layers.Dense(units=classes, activation='softmax')(x)


        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )


        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=str(self.config.updated_base_model_path), model=self.full_model)
    


In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()  # Fixed method name
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    print("Model preparation completed successfully!")
except Exception as e:
    print(f"Error occurred: {e}")
    raise e

[2025-07-12 00:35:02,503:INFO:common:yaml file config\config.yaml loaded successfully]
[2025-07-12 00:35:02,537:INFO:common:yaml file params.yaml loaded successfully]
[2025-07-12 00:35:02,541:INFO:common:created directory at :artifacts]
{'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}
[2025-07-12 00:35:02,541:INFO:common:created directory at :artifacts/prepare_base_model]
[2025-07-12 00:35:05,312:WARNING:saving_utils:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []     